# Working with categorical data: land cover

This example builds upon the sample data for São Miguel introduced [here](basic_usage.ipynb) to demonstrate the use of exactextract with categorical land cover data.

The following plot shows a sample of the [CORINE 2018 landcover dataset](https://land.copernicus.eu/en/products/corine-land-cover) that is distributed with exactextractr.

In [ ]:
import geopandas as gpd
import rasterio
import rasterio.plot
from matplotlib import pyplot

landcov = 'sao_miguel/clc2018_v2020_20u1.tif'
concelhos = 'sao_miguel/concelhos.gpkg'

fig, ax = pyplot.subplots()

with rasterio.open(landcov) as r:
    rasterio.plot.show(r, ax=ax, cmap='tab20c')
    
admin = gpd.read_file(concelhos)
admin.plot(ax=ax, facecolor='none', edgecolor='black')

## Summarizing land cover classifications

One of the most basic questions we might ask is which land cover type is predominant in each _concelho_. We can do this with the built-in `mode` summary operation. The `minority` and `variety` operations are also applicable to categorical data and provide the least-common classification and number of distinct classifications, respectively.

In [ ]:
from exactextract import exact_extract

df = exact_extract(landcov, concelhos, ["variety", "mode", "minority"], include_cols='name', output='pandas')
display(df)

If needed, we can load the attribute table distributed with the landcover dataset and replace the numeric codes above with their descriptions.

In [ ]:
import geopandas as pd

classes = gpd.read_file('sao_miguel/clc2018_v2020_20u1.tif.vat.dbf').set_index('Value')
for col in ('mode', 'minority'):
    df[col] = df[col].map(classes['LABEL3'])
display(df)

## Calculating the fraction of each land cover type

If we want more detailed information, the `unique` operation provides an array of distinct landcover types within each polygon.
The `frac` operation provides a matching array with fraction of the polygon's Cartesian area that is covered by each type.

In [ ]:
df = exact_extract(landcov, concelhos, ["unique", "frac"], include_cols='name', output='pandas')
display(df)

To join these codes with the description, we can unnest the data using the pandas ``explode`` function and then remap the fields as done previously.

In [ ]:
df = df.explode(['unique', 'frac'])
df['unique'] = df['unique'].map(classes['LABEL3'])
display(df)

## Condensed output with ``frac_as_map``

If we are working with JSON output, it is also possible to view the same information in a map format:

In [ ]:
exact_extract(landcov, concelhos, 'frac', include_cols='name', output='geojson', output_options={'frac_as_map': True})[:2]

## Summarizing population land cover

One extension of the analysis above is to see which land covers are associated with human population in a given concelho. Is the population primary urban or rural?

As described in the [basic usage example](basic_usage.ipynb), the population density raster provides the most robust results in the presence of partially-covered pixels.

We are able to perform this analysis because the CORINE sample distributed with exactextractr has been reprojected from its native Lambert Equal Area projection into geographic coordinates consistent with GPW. Otherwise, working with multiple rasters in different projections requires transformation to a common grid using tools such as GDAL.

Very little about the call to `exact_extract` requires changing to incorporate population. We swap `weighted_frac` for `frac` and set `weights = pop_density`:

In [ ]:
pop_density = 'sao_miguel/gpw_v411_2020_density_2020.tif'

df = exact_extract(landcov, concelhos, ["unique", "weighted_frac"], weights=pop_density, include_cols='name', output='pandas')
df = df.explode(['unique', 'weighted_frac']).astype({'weighted_frac':'float64'})
df['unique'] = df['unique'].map(classes['LABEL3'])
df.sort_values('weighted_frac', ascending=False).drop_duplicates('name')

Looking at the highest-population land cover type in each concelho, we can can see that the western/central concelhos of Lagoa, Ponta Delgada, Ribeira Grande, and Vila Franca do Campo have a more urban population than Nordeste or Povoação to the east.